## Thông tin nhóm

**Thành viên thứ 1:**

- Họ và tên: Trần Trung Hiếu
- MSSV: 1712442

**Thành viên thứ 2:**
- Họ và tên: Lê Long Hồ
- MSSV: 1712447

**Thành viên thứ 3:**
- Họ và tên: Nguyễn Đình Thiên Phúc
- MSSV: 18120144

___

## Import

In [2]:
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
pd.set_option('display.max_rows', 500)

---

## Thu thập dữ liệu

TODO

---

## Khám phá dữ liệu

In [6]:
movie_df = pd.read_csv('movie_final.csv')
movie_df.sample(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
5047,https://www.themoviedb.org/movie/183011,Justice League: The Flashpoint Paradox\n(2013),78.0,poster_final/183011.jpg,PG-13,When time travel allows a past wrong to be rig...,Fight the future.,"Phim Giả Tượng, Phim Khoa Học Viễn Tưởng, Phim...",NaN,en
138,https://www.themoviedb.org/tv/62046,Scream Queens\n(2015),79.0,poster_final/62046.jpg,TV-14,The super-charged comedy-horror series is a mo...,Pretty evil.,"Phim Bí Ẩn, Phim Hình Sự, Phim Hài, Phim Chính...",NaN,en
2282,https://www.themoviedb.org/tv/41727,Banshee\n(2013),79.0,poster_final/41727.jpg,TV-MA,Banshee is an American drama television series...,Small town. Big secrets.,"Phim Hình Sự, Phim Chính Kịch",NaN,en
3969,https://www.themoviedb.org/movie/659994,Rams\n(2020),66.0,poster_final/659994.jpg,PG,"In remote Western Australia, two estranged far...","Where there's a wool, there's a way","Phim Chính Kịch, Phim Hài",NaN,en
2699,https://www.themoviedb.org/movie/46429,Những Tay Súng Diệt Quỷ\n(2012),41.0,poster_final/46429.jpg,R,"Bộ phim kể về một tay súng bí ẩn, Aman, là con...",NaN,"Phim Hành Động, Phim Kinh Dị, Phim Miền Tây",NaN,vi


In [27]:
n_row, n_col = movie_df.shape
print("Dữ liệu có số dòng:", n_row)
print("Dữ liệu có số cột:", n_col)

Dữ liệu có số dòng: 5598
Dữ liệu có số cột: 10


Mỗi dòng trong tập dữ liệu thể hiện vài thông tin của một bộ phim, bộ phim đó có thể là phim truyền hình hoặc phim điện ảnh. Các dòng này đều có ý nghĩa tương đương nhau. 

## Đưa ra câu hỏi cần trả lời

TODO

---

## Khám phá dữ liệu

TODO

## Tiền xử lí - tách các tập dữ liệu

TODO

---

## Khám phá dữ liệu - tập huấn luyện

## Tiền xử lí

In [19]:
movie_df.language.value_counts()

en    4304
vi    1224
lt      31
fi      10
tl       5
pt       4
cy       4
ro       3
es       2
de       1
cs       1
af       1
fr       1
it       1
sk       1
id       1
Name: language, dtype: int64

In [25]:
movie_df[movie_df.score != 0.0]

F:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
606,https://www.themoviedb.org/tv/46101,Senhora do Destino\n(2004),95.0,poster_final/46101.jpg,L,Senhora do Destino foi uma telenovela brasilei...,NaN,"Phim Chính Kịch, Soap, Phim Gia Đình, Soap",NaN,pt
1687,https://www.themoviedb.org/tv/75380,Tô de Graça\n(2017),85.0,poster_final/75380.jpg,12,"Na nova série de comédia Tô de Graça, Rodrigo ...",NaN,Phim Hài,NaN,pt


In [14]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              5598 non-null   object 
 1   title            5594 non-null   object 
 2   score            5594 non-null   float64
 3   posterImagePath  5326 non-null   object 
 4   certification    4177 non-null   object 
 5   overview         5594 non-null   object 
 6   tagline          2522 non-null   object 
 7   genres           5230 non-null   object 
 8   trailerUrl       79 non-null     object 
 9   language         5594 non-null   object 
dtypes: float64(1), object(9)
memory usage: 437.5+ KB


In [18]:
movie_df.certification.value_counts()

R             622
TV-14         518
PG-13         455
PG            407
TV-MA         407
TV-PG         205
G             173
12            169
NR            167
15            138
16            111
TV-Y7          94
TV-G           72
U              62
18             49
14             47
L              44
16+            29
10             25
0              25
6              23
18+            20
T              18
12+            17
MA15+          16
保護級            15
TV-Y           15
普遍級            14
AL             14
6+             12
7              12
輔12級           10
M               9
NC-17           9
UA              9
APTA            9
11              8
M/12            7
NC16            6
9               6
C13             6
15세 이상 관람가      6
12세 이상 관람가      5
ALL             5
A               4
PG13            4
C18             4
0+              4
C8              4
輔15級            4
12A             4
14+             4
19              4
PG12            3
17+             3
15세이상관람가  

## Tiền xử lí - tập validation

TODO

---

## Mô hình hóa dữ liệu

### Hyperparameter tuning

TODO

### Đánh giá mô hình

TODO

---

## Kết luận

TODO